## Mesh Extraction
Run the following code to extract a mesh given a .pth file name. Change the pth_file variable to that of the model you would like to create a mesh for.

In [2]:
import mcubes
import trimesh
import torch
import open3d as o3d
import numpy as np

import evaluate as eval

from ml_helpers import load_checkpoint
from model import Nerf

In [3]:
device = 'cuda'
pth_file = 'experiments/suzanne/set0/models/M0.pth'
#pth_file = 'experiments/monkey_3_15aug_2/models/M9.pth'
model = torch.load(pth_file).to(device)


In [159]:
# # OR
# device = 'cuda'
# nb_epochs = 5
# lr = 1e-3
# gamma = 0.5
# tn = 1 # these depend on the dataset tf/tn
# tf = 10
# nb_bins = 100
# model_name = 'monkey_3_big_aug'
# ckpt_name = '/ckpt100.pth'
# checkpoint_path = 'experiments/'+model_name

# model = Nerf(hidden_dim=256).to(device)
# optimizer = torch.optim.Adam(model.parameters(),lr=lr)
# # Every 5 epochs/images in online learning case, we multiply the learning rate by gamma (1/2)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,10],gamma=gamma)

# ckpt_file = load_checkpoint(checkpoint_path+ckpt_name, model, optimizer, scheduler)

In [160]:

# model.load_state_dict(ckpt_file['model_state_dict'])
# optimizer.load_state_dict(ckpt_file['optimizer_state_dict'])
# model.eval();


In [4]:
N = 100
scale = 1.5

x = torch.linspace(-scale, scale, N)
y = torch.linspace(-scale, scale, N)
z = torch.linspace(-scale, scale, N)

x, y, z = torch.meshgrid((x, y, z))

xyz = torch.cat((x.reshape(-1, 1),
                 y.reshape(-1, 1),
                 z.reshape(-1, 1)), dim=1)

/home/eherrin@ad.ufl.edu/anaconda3/envs/nerf3Dchange/lib/python3.12/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1718580525958/work/aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
with torch.no_grad():
    _, density = model.forward(xyz.to(device), torch.zeros_like(xyz).to(device))
    
density = density.cpu().numpy().reshape(N, N, N)

In [6]:
vertices, triangles = mcubes.marching_cubes(density, .3* np.mean(density))
mesh = trimesh.Trimesh(vertices / N, triangles)
mesh.show()

In [ ]:
import numpy as np
import pyvista as pv
#pv.set_jupyter_backend('client')

# Example: Assuming sample_variance is a 100x100x100 ndarray and threshold is the threshold scalar value

# Set a threshold scalar value
threshold = 5  # Example threshold value

# # Find the coordinates of points above the threshold
above_threshold_mask = density > threshold
x_coords, y_coords, z_coords = np.where(above_threshold_mask)

# Extract the values of points above the threshold
values = density[above_threshold_mask]
np.save('monkey_mask.npy', (x_coords, y_coords, z_coords))


# Create a PyVista Plotter
plotter = pv.Plotter(title="Densities")

# Add the points to the plotter
plotter.add_points(np.column_stack((x_coords, y_coords, z_coords)), scalars=values, cmap="inferno")

# Show the plotter window
plotter.show()

In [7]:
# # Convert the mesh to a point cloud
point_cloud = trimesh.points.PointCloud(vertices=vertices)


In [9]:
point_cloud.export('suzanne_points.ply')

ValueError: ('%s exporter not available!', 'pcd')